In [7]:
import csv, sys, os, math
import pandas as pd
import numpy as np
from numpy import argmax
import random
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
from sklearn.model_selection import train_test_split 
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score,roc_curve,auc  
import matplotlib.pyplot as plt
import matplotlib
from matplotlib import pyplot
from functools import reduce
from pathlib import Path
from sklearn.model_selection import StratifiedKFold

from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from boruta import BorutaPy

In [8]:
workdir = os.path.abspath('.')
#cli_data = pd.read_csv('%s/BRCA_clinical.tsv'%workdir,sep='\t')
exp_data = pd.read_csv('%s/BRCA.sampleMap_HiSeqV2'%workdir,sep='\t')
gene_name = exp_data['sample']
#gene_name.to_csv('%s/gene_name.txt'%workdir,sep='\t') #写入文件保存
exp_data = exp_data.drop(labels = 'sample',axis=1).transpose()
exp_data.columns = gene_name

ylable= pd.DataFrame(pd.DataFrame(exp_data.index)[0].str.split('-').str[-1])
exp_data['ylable'] = np.where(ylable[0].str.contains("0"), 1, 0)
exp_data.to_csv('%s/BRCA.txt'%workdir,sep='\t',index='sample')

x = exp_data.iloc[:,0:20531]
x = x.drop(columns = 'ylable')
y = exp_data.iloc[:,-1].astype(int)


# RF-Boruta

In [11]:
importance_list = []
x_train,x_test,y_train,y_test = train_test_split(x, y, test_size = 0.2,random_state = 32, stratify=y)
rf = RandomForestClassifier(n_jobs=-1, class_weight=None, max_depth=7, random_state=0)
feat_selector = BorutaPy(rf, n_estimators='auto', verbose=2, random_state=0)
feat_selector.fit(x_train.copy().values, y_train)

In [12]:
importance_list.append(feat_selector.ranking_)
df_importance = pd.DataFrame(importance_list).fillna(0)
df_importance = df_importance.rename(columns = gene_name)
df_result = df_importance.mean(axis =0).sort_values(ascending= 0)
max_importance = df_result[0]

In [13]:
df_importance

ARHGEF10L  HIF3A  RNF17  RNF10  RNF11  RNF13  GTF2IP1   REM1  MTVR2  \
0       9335      1  17554   5009  12834   6974    12834  12834  10699   

   RTN4RL2  ...  TULP2  NPY5R  GNGT2  GNGT1  TULP3  PTRF  BCL6B  GSTK1  SELP  \
0      123  ...   6974   1542  10699  17554   7806     1   9335   4430  6247   

   SELS  
0  6974  

[1 rows x 20530 columns]

In [14]:
df_result

OR10AD1        17554.0
SNORD116-15    17554.0
SNORD116-10    17554.0
ATG9A          17554.0
PFN4           17554.0
                ...   
KCNE1              1.0
TRPM3              1.0
PROC               1.0
CAT                1.0
CNTNAP3            1.0
Length: 20530, dtype: float64

In [16]:
list(df_result.index)

['OR10AD1',
 'SNORD116-15',
 'SNORD116-10',
 'ATG9A',
 'PFN4',
 'SNORD116-13',
 'PFN2',
 'CALML4',
 'VENTXP1',
 'SNORA38',
 'TMEM194A',
 'SPANXN3',
 'IFT74',
 'ARPP21',
 'SNORD91A',
 'RBMY1E',
 'OR8D2',
 'OR8D1',
 'MYL10',
 'NT5DC1',
 'OR4B1',
 'OR8D4',
 'SPRR3',
 'SNORD116-14',
 'LOC347376',
 'NBEA',
 'RAP2A',
 'TPM4',
 'CELA3B',
 'CELA3A',
 'OR1A2',
 'FBXW4',
 'C22orf30',
 'C22orf33',
 'RPS5',
 'EPSTI1',
 'LOC678655',
 'CSF1R',
 'LIMS3-LOC440895',
 'COX16',
 'BET1',
 'LOC339568',
 'tAKR',
 'PRSS48',
 'PPA2',
 'C9orf6',
 'ANKRD13C',
 'ANKRD13B',
 'LOC158572',
 'ANKRD13D',
 'SPRR4',
 'NPHS2',
 'C10orf53',
 'GSTTP1',
 'ZNF100',
 'LOC340508',
 'ARRDC5',
 'SNX9',
 'SBNO1',
 'NCRNA00171',
 'C5orf40',
 'C5orf46',
 'TMEM209',
 'GSTTP2',
 'XKR3',
 'SNORD116-16',
 'XKR9',
 'PRPF4',
 'TADA3',
 'RNF168',
 'SNORD90',
 'NTAN1',
 'UGT1A8',
 'UGT1A9',
 'UGT1A6',
 'UGT1A1',
 'SNORD95',
 'LOC100271832',
 'GPHB5',
 'ELK4',
 'LCN15',
 'DEFB119',
 'AP1G1',
 'TTTY4C',
 'CYP2D6',
 'DEFB113',
 'DEFB112',
 '

In [17]:
name=['Boruta_rf']
name2=range(1,len(df_result.index)+1)
test=pd.DataFrame(columns=name,index=name2,data=df_result.index)
print(test)

         Boruta_rf
1          OR10AD1
2      SNORD116-15
3      SNORD116-10
4            ATG9A
5             PFN4
...            ...
20526        KCNE1
20527        TRPM3
20528         PROC
20529          CAT
20530      CNTNAP3

[20530 rows x 1 columns]


In [18]:
test.to_csv('Boruta_rf特征排序结果.csv')

# KNN-RF-Boruta

### 用Boruta拟合RF前，先用KNN进行一次特征筛选

In [17]:
%%time
cv = StratifiedKFold(n_splits=2)
importance_list = []

for train_index, test_index in cv.split(x,y):
        x_train = x.iloc[train_index]
        y_train = y.iloc[train_index]
        
        knn = KNeighborsClassifier(n_neighbors=3)
        k_best = SelectKBest(score_func=f_classif, k=5)
        x_train_best = k_best.fit_transform(x_train, y_train)
        knn.fit(x_train_best, y_train)

        rf = RandomForestClassifier(n_jobs=-1, class_weight=None, max_depth=7, random_state=0)
        feat_selector = BorutaPy(rf, n_estimators='auto', verbose=2, random_state=0)
        feat_selector.fit(x_train_best, y_train)

D:\Conda\anConda\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:112: UserWarning: Features [  144   214   283   284   297   321   365   565   732   819   823   824
   825   839  1077  1100  1188  1234  1255  1256  1257  1259  1338  1363
  1367  1399  1425  1430  1461  1619  1655  1743  1765  1766  1891  1993
  1994  2003  2031  2032  2037  2224  2225  2231  2236  2248  2391  2409
  2476  2690  2691  2693  2698  2699  2733  2889  2903  2944  3039  3048
  3049  3076  3079  3206  3273  3284  3288  3425  3463  3487  3895  3994
  4036  4050  4051  4052  4115  4135  4933  5009  5033  5242  5299  5371
  5418  5439  5606  5607  5609  5610  5648  5755  6030  6049  6051  6118
  6125  6147  6323  6324  6533  6541  6825  6852  6855  6856  6860  7207
  7218  7292  7357  7403  7420  7421  7440  7602  7800  7857  7858  8000
  8025  8107  8276  8353  8565  9029  9138  9200  9243  9305  9365  9485
  9621  9625  9666  9755  9969 10262 10315 10458 10464 10502 10527 10528
 10565 1058

AttributeError: module 'numpy' has no attribute 'int'.
`np.int` was a deprecated alias for the builtin `int`. To avoid this error in existing code, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
The aliases was originally deprecated in NumPy 1.20; for more details and guidance see the original release note at:
    https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations